In [55]:
import pandas as pd
import numpy as np
import csv
pd.options.mode.chained_assignment = None 

Load the CSV in as a DataFrame

In [67]:
df = pd.read_csv('raw_data/2020.csv')
df.head()

,Rk,Nation,# Players,Min,List
0,1,eng England,249,274172.0,Harry Kane Patrick Bamford Ollie Watkins Jamie...
1,2,fr France,32,38786.0,Hugo Lloris Neal Maupay Lucas Digne Alexandre ...
2,3,es Spain,30,39874.0,Pablo Fornals Vicente Guaita Rodri Adama Traor...
3,4,ie Republic of Ireland,29,22244.0,David McGoldrick Enda Stevens John Egan Dara O...
4,5,br Brazil,26,41438.0,Roberto Firmino Richarlison Raphinha Ederson G...


We can see that the "Nation" column as the country abbreviation followed by the country name, so we can split that out into 2 separate columns, by separating on the first space (' ') value. 

In [68]:
abb = df['Nation'].apply(lambda x: x.split(' ', 1)[0])
df['Nation'] = df['Nation'].apply(lambda x: x.split(' ',1)[1])
df['Abbreviation'] = abb

In [69]:
df.head()

,Rk,Nation,# Players,Min,List,Abbreviation
0,1,England,249,274172.0,Harry Kane Patrick Bamford Ollie Watkins Jamie...,eng
1,2,France,32,38786.0,Hugo Lloris Neal Maupay Lucas Digne Alexandre ...,fr
2,3,Spain,30,39874.0,Pablo Fornals Vicente Guaita Rodri Adama Traor...,es
3,4,Republic of Ireland,29,22244.0,David McGoldrick Enda Stevens John Egan Dara O...,ie
4,5,Brazil,26,41438.0,Roberto Firmino Richarlison Raphinha Ederson G...,br


That worked, and now we can rearrange the columns. The List column has names without any separators between them - unfortunately, we can't fix this. We have no way of knowing which names are a first name and a surname, and which names have multiple spaces ("Van Persie") and which players go by one name ("Richarlison"). However, we will keep them as having a few recognizable names may help us or be of interest

In [70]:
df = df[['Rk', 'Nation', 'Abbreviation', '# Players', 'Min', 'List']]
df.rename(columns={'Min': 'Minutes', '# Players': 'Count', 'Rk': 'Rank'}, inplace=True)
df.head()

,Rank,Nation,Abbreviation,Count,Minutes,List
0,1,England,eng,249,274172.0,Harry Kane Patrick Bamford Ollie Watkins Jamie...
1,2,France,fr,32,38786.0,Hugo Lloris Neal Maupay Lucas Digne Alexandre ...
2,3,Spain,es,30,39874.0,Pablo Fornals Vicente Guaita Rodri Adama Traor...
3,4,Republic of Ireland,ie,29,22244.0,David McGoldrick Enda Stevens John Egan Dara O...
4,5,Brazil,br,26,41438.0,Roberto Firmino Richarlison Raphinha Ederson G...


I want to change the 'Minutes' column to an integer, but there are NaN values which are giving me errors, so lets remove those first. If we have a NaN in the minutes column, we can reasonably assume that is due to 0 minutes being played by that nation, so lets change those. 

In [76]:
print(df['Minutes'].isnull().sum())
df.tail(10)

5


,Rank,Nation,Abbreviation,Count,Minutes,List
56,57,IR Iran,ir,1,528.0,Alireza Jahanbakhsh
57,58,Guinea,gn,1,520.0,Naby Keïta
58,59,Mauritania,mr,1,315.0,Aboubakar Kamara
59,60,Bosnia and Herzegovina,ba,1,90.0,Sead Kolašinac
60,61,Canada,ca,1,9.0,Theo Corbeanu
61,62,Albania,al,1,NaN,Meritan Shabani
62,63,Bulgaria,bg,1,NaN,Sylvester Jasper
63,64,Ecuador,ec,1,NaN,Moisés Caicedo
64,65,Romania,ro,1,NaN,Florin Andone
65,66,Thailand,th,1,NaN,Thanawat Suengchitthawon


We have 5 NaN values which should all be at the tail/end. We can see they are all present, so we can just change those to 0, and then convert the column to integers. 

In [90]:
df = df.fillna(0)
df['Minutes'] = df['Minutes'].astype('int64')
print(df.dtypes)

Rank             int64
Nation          object
Abbreviation    object
Count            int64
Minutes          int64
List            object
dtype: object


In [92]:
df.tail(10)

,Rank,Nation,Abbreviation,Count,Minutes,List
56,57,IR Iran,ir,1,528,Alireza Jahanbakhsh
57,58,Guinea,gn,1,520,Naby Keïta
58,59,Mauritania,mr,1,315,Aboubakar Kamara
59,60,Bosnia and Herzegovina,ba,1,90,Sead Kolašinac
60,61,Canada,ca,1,9,Theo Corbeanu
61,62,Albania,al,1,0,Meritan Shabani
62,63,Bulgaria,bg,1,0,Sylvester Jasper
63,64,Ecuador,ec,1,0,Moisés Caicedo
64,65,Romania,ro,1,0,Florin Andone
65,66,Thailand,th,1,0,Thanawat Suengchitthawon


The DataFrame looks good, so we can save to a CSV file and prepare to clean our other files. 

In [93]:
df.to_csv('cleaned_data/2020.csv', index=False)

Now after visually checking that our other CSV files look similar, we can automate the cleaning of those files. 

In [101]:
def clean_data(fname):
    df = pd.read_csv('raw_data/{}.csv'.format(fname))
    abb = df['Nation'].apply(lambda x: x.split(' ', 1)[0])
    df['Nation'] = df['Nation'].apply(lambda x: x.split(' ',1)[1])
    df['Abbreviation'] = abb
    df = df[['Rk', 'Nation', 'Abbreviation', '# Players', 'Min', 'List']]
    df.rename(columns={'Min': 'Minutes', '# Players': 'Count', 'Rk': 'Rank'}, inplace=True)
    df = df.fillna(0)
    df['Minutes'] = df['Minutes'].astype('int64')
    return df

In [102]:
for i in range(1992, 2023):
    fname = str(i)
    df = clean_data(fname)
    df.to_csv('cleaned_data/{}.csv'.format(fname), index=False)
    print('Saved file {}.csv to cleaned_data'.format(fname))


Saved file 1992.csv to cleaned_data
Saved file 1993.csv to cleaned_data
Saved file 1994.csv to cleaned_data
Saved file 1995.csv to cleaned_data
Saved file 1996.csv to cleaned_data
Saved file 1997.csv to cleaned_data
Saved file 1998.csv to cleaned_data
Saved file 1999.csv to cleaned_data
Saved file 2000.csv to cleaned_data
Saved file 2001.csv to cleaned_data
Saved file 2002.csv to cleaned_data
Saved file 2003.csv to cleaned_data
Saved file 2004.csv to cleaned_data
Saved file 2005.csv to cleaned_data
Saved file 2006.csv to cleaned_data
Saved file 2007.csv to cleaned_data
Saved file 2008.csv to cleaned_data
Saved file 2009.csv to cleaned_data
Saved file 2010.csv to cleaned_data
Saved file 2011.csv to cleaned_data
Saved file 2012.csv to cleaned_data
Saved file 2013.csv to cleaned_data
Saved file 2014.csv to cleaned_data
Saved file 2015.csv to cleaned_data
Saved file 2016.csv to cleaned_data
Saved file 2017.csv to cleaned_data
Saved file 2018.csv to cleaned_data
Saved file 2019.csv to clean

Create one more CSV file to combine all the previous files into one. This may help when looking at time series data.

In [136]:
df = pd.read_csv('cleaned_data/1992.csv')
df['Year'] = 1992
for i in range(1993, 2023):
    add_df = pd.read_csv('cleaned_data/{}.csv'.format(str(i)))
    add_df['Year'] = i
    df = pd.concat([df, add_df])
df = df[['Year', 'Rank', 'Nation', 'Abbreviation', 'Count', 'Minutes', 'List']]

One more thing to do is get a list of countries with their respective continent so we can examine nationalities by continent in the future. I downloaded a CSV file from https://gist.github.com/stevewithington. 

In [194]:
continents = pd.read_csv('raw_data/country_and_continents.csv')
continents.rename(columns={'Two_Letter_Country_Code': 'Abbreviation'}, inplace=True)
continents.head()

,Continent_Name,Continent_Code,Country_Name,Abbreviation,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


However I do have to change it a bit since some countries, at first glance the UK countries, don't exist in the list. I also notice UK countries have 3 letter abbreviations in our DataFrame, so lets check for any other countries with unusual abbreviations

In [125]:
three_letter = []
for i, row in df.iterrows():
    if len(row['Abbreviation'])>2:
        if row.Nation not in three_letter:
            three_letter.append(row.Nation)
three_letter

['England', 'Scotland', 'Wales', 'Northern Ireland', 'and Montenegro']

As expected, the UK countries are outliers, but so is "and Montenegro". After inspecting the data, it looks like the country code is "SERBIA" and "and Montenegro" is the nation name. While this could have been a data insertion error, there may be more to it - since, in the 90's and early 2000's, Serbia & Montenegro played under the Yugoslavian banner. Only 1 player is from "and Montenegro" who, after a little research, appears to be Australian-born. We wouldn't be able to say which of Serbia or Montenegro he would pledge allegiance too, but further inspection of the data shows other Serbian players in the 90's. The Serbian team did not come about until 2006, so we can put Sasa Ilic in that bin. 

In [139]:
df.replace('and Montenegro', 'Serbia', inplace=True)
df.replace('SERBIA', 'RS', inplace=True)
df['Abbreviation'] = df['Abbreviation'].str.upper()

In [129]:
join = pd.merge(df, continents[['Continent_Name', 'Continent_Code', 'Abbreviation']], on='Abbreviation', how='left', indicator=True)

In [172]:
mask = continents.Country_Name.duplicated(keep=False)
continents[mask]

,Continent_Name,Continent_Code,Country_Name,Abbreviation,Three_Letter_Country_Code,Country_Number
8,Europe,EU,"Azerbaijan, Republic of",AZ,AZE,31.0
9,Asia,AS,"Azerbaijan, Republic of",AZ,AZE,31.0
16,Europe,EU,"Armenia, Republic of",AM,ARM,51.0
17,Asia,AS,"Armenia, Republic of",AM,ARM,51.0
58,Europe,EU,"Cyprus, Republic of",CY,CYP,196.0
59,Asia,AS,"Cyprus, Republic of",CY,CYP,196.0
83,Europe,EU,Georgia,GE,GEO,268.0
84,Asia,AS,Georgia,GE,GEO,268.0
116,Europe,EU,"Kazakhstan, Republic of",KZ,KAZ,398.0
117,Asia,AS,"Kazakhstan, Republic of",KZ,KAZ,398.0


I create a join but turn on the indicator to check if we have any more unseen issues with the indicator, and we do - XC (Independent States) and XK (Kosovo) are both not in the continent list. We will add those to the continent list along with the UK countries. I also see that several countries have multiple entries, mainly due to being on the border of 2 continents. Since the North America abbreviation is "NA", pandas reads it from the CSV as a NaN value, so we will have to make sure to read the CSV in correctly in the future. 

In [195]:
continents.loc[len(continents)] = ['Europe', 'EU', 'England', 'ENG', '', '']
continents.loc[len(continents)] = ['Europe', 'EU', 'Wales', 'WLS', '', '']
continents.loc[len(continents)] = ['Europe', 'EU', 'Scotland', 'SCT', '', '']
continents.loc[len(continents)] = ['Europe', 'EU', 'Northern Ireland', 'NIR', '', '']
continents.loc[len(continents)] = ['Europe', 'EU', 'Kosovo', 'XK', '', '']
continents.loc[len(continents)] = ['Europe', 'EU', 'Commonwealth of Independent States', 'XC', '', '']
continents['Continent_Code'] = continents['Continent_Code'].fillna(value='NA')
continents.drop([9, 17, 59, 84, 116, 172, 192, 235], inplace=True)
continents.tail(8)

,Continent_Name,Continent_Code,Country_Name,Abbreviation,Three_Letter_Country_Code,Country_Number
260,Asia,AS,United Nations Neutral Zone,XD,NaN,NaN
261,Asia,AS,Spratly Islands,XS,NaN,NaN
262,Europe,EU,England,ENG,,
263,Europe,EU,Wales,WLS,,
264,Europe,EU,Scotland,SCT,,
265,Europe,EU,Northern Ireland,NIR,,
266,Europe,EU,Kosovo,XK,,
267,Europe,EU,Commonwealth of Independent States,XC,,


Finally we can join the two tables and check that we haven't lost any rows or added duplicates, and then save to a new CSV file. 

In [204]:
join = pd.merge(df, continents[['Continent_Name', 'Continent_Code', 'Abbreviation']], on='Abbreviation', how='inner')
join.rename(columns={'Continent_Name': 'Continent', 'Continent_Code': 'Con_Code', 'Abbreviation': 'Nat_Code'}, inplace=True)
print(len(join) == len(df))
print(join.head())

True
   Year  Rank   Nation Nat_Code  Count  Minutes  \
0  1992     1  England      ENG    363   619773   
1  1993     1  England      ENG    367   630123   
2  1994     1  England      ENG    350   608645   
3  1995     1  England      ENG    312   475290   
4  1996     1  England      ENG    286   416543   

                                                List Continent Con_Code  
0  Brian Deane Matt Le Tissier Teddy Sheringham L...    Europe       EU  
1  Andy Cole Chris Sutton Matt Le Tissier Alan Sh...    Europe       EU  
2  Alan Shearer Matt Le Tissier Robbie Fowler Les...    Europe       EU  
3  Alan Shearer Robbie Fowler Les Ferdinand Teddy...    Europe       EU  
4  Alan Shearer Ian Wright Robbie Fowler Les Ferd...    Europe       EU  


In [205]:
join.to_csv('cleaned_data/all_years.csv', index=False)